In [1]:
import ee
import geemap
import pandas as pd
import geopandas as gpd
import rasterio as rio
import matplotlib.pyplot as plt
import datetime
import pyproj

In [2]:
# Read pipe network and leaks loc
pipes_raw = gpd.read_file("../data/shp/Tuberias.shp")
leaks_raw = pd.read_excel("../data/raw/Ordenes Fugas Ejecutadas Dic2022 XY.xlsx")

In [3]:
# Define geometry from coords in leaks dataset
leaks_clean = leaks_raw.copy()
leaks_clean["geometry"] = gpd.points_from_xy(leaks_clean["X"], leaks_clean["Y"])

# Create gepandas gpd 
leaks_clean = gpd.GeoDataFrame(leaks_clean, geometry = "geometry")
pipes_clean = pipes_raw.copy()

In [4]:
pipes_clean.crs

<Compound CRS: COMPD_CS["WGS 84 / UTM zone 13N + NAVD88 height",P ...>
Name: WGS 84 / UTM zone 13N + NAVD88 height
Axis Info [cartesian|vertical]:
- [east]: Easting (metre)
- [north]: Northing (metre)
- [up]: Gravity-related height (metre)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
Sub CRS:
- WGS 84 / UTM zone 13N
- NAVD88 height

In [5]:
# leaks_clean = leaks_clean.set_crs("WGS 84")
leaks_clean.crs

In [ ]:
#pipes_clean.to_crs(4326)
#leaks_clean.to_crs(4326)

# Download 10x10 patch from GEE

In [6]:
ee.Initialize()
Map = geemap.Map()

In [7]:
# Coords of leaks
leak_lat = leaks_clean["Y"][0]
leak_lon = leaks_clean["X"][0]

# Transform to degrees
transformer = pyproj.Transformer.from_crs("epsg:32613", "epsg:4326")
leak_lon, leak_lat = transformer.transform(leak_lon, leak_lat)

# Point of leak
leak_poi = ee.Geometry.Point([leak_lat, leak_lon])

# Date of leak
end_date = leaks_clean["FECHALEGALIZACION"][0]
start_date = end_date - datetime.timedelta(10)

end_date = str(end_date)[:10]
start_date = str(start_date)[:10]

print((start_date, end_date))
print((leak_lat, leak_lon))

('2022-11-27', '2022-12-07')
(-102.27943290775767, 21.935910837898607)


In [8]:
collection = ee.ImageCollection("COPERNICUS/S2_SR")\
                .filterBounds(leak_poi)\
                .filterDate(ee.Date(start_date), ee.Date(end_date))

print("Images: ", collection.size().getInfo())

collection_list = collection.toList(collection.size())

Images:  4


In [10]:
bands = ["B4", "B3", "B2"]

img0 = ee.Image(collection_list.get(0))
img0 = img0.select(bands)

# Clip patch size
img0 = img0.clip(leak_poi.buffer(200).bounds())


viz_params = {
    "bands": ["B4", "B3", "B2"],
    "min": 0,
    "max": 3000
}

# Map.setCenter(leak_lat, leak_lon)
Map.addLayer(img0, viz_params, "img0")
Map.centerObject(img0)
Map

Map(bottom=3670659.0, center=[21.929391291441927, -102.28211402893068], controls=(WidgetControl(options=['posi…